In [ ]:
!pip install -Uqq fastai duckduckgo_search nbdev
# nbdev is needed for notebook2script 

In [ ]:
#|default_exp app

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=100):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
from fastdownload import download_url
dest = 'squirrel.jpg'

urls = search_images('squirrel', max_images=1)
urls[0]

download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('bird', max_images=1)[0], 'bird.jpg', show_progress=False)
Image.open('bird.jpg').to_thumb(256,256)

In [ ]:
download_url(search_images('forest', max_images=1)[0], 'forest.jpg', show_progress=False)
Image.open('forest.jpg').to_thumb(256,256)

In [ ]:
from time import sleep

searches = 'squirrel', 'bird', 'forest'
path = Path('nature_images')

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)
    
print('Image search complete!')

# Train

First, removing images that failed to download

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Create training and validation sets with images

In [ ]:
nature = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5), batch_tfms=aug_transforms()
)

dls = nature.dataloaders(path)

dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

# Training to clean the data

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
from fastai.vision.widgets import *

cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

# Exporting the model

In [ ]:
learn.export('model.pkl')

# Code for the interface
Loading the model and using it

In [ ]:
#|export
learn = load_learner('model.pkl')

In [ ]:
#|export
categories = ('bird', 'forest', 'squirrel')

def classify_image(img):
    pred,idx,probs = learn.predict(img)
    return dict(zip(categories, map(float,probs)))

> Results come as bird :: forest :: squirrel
>
> I wonder if the order of categories matter.

It turns out it does. Need to investigate where that is defined.

# Exporting notebook to app.py

In [ ]:
%notebook -e testnbdev.ipynb

In [ ]:
from nbdev.export import nb_export
nb_export('testnbdev.ipynb', '.')

In [ ]:
!cat app.py